<a href="https://colab.research.google.com/github/AI-Front/ChatBots/blob/master/4-Question_Answering/SQuaD_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQuAD System for English

**S**tanford **Q**uestion **A**nswering **D**ataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

SQuAD2.0 combines the 100,000 questions in SQuAD1.1 with over 50,000 unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. To do well on SQuAD2.0, systems must not only answer questions when possible, but also determine when no answer is supported by the paragraph and abstain from answering.

The [Dataset](https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/)

The [Paper](http://arxiv.org/abs/1806.03822)

Example:
![squad_example.png](https://github.com/AI-Front/ChatBots/blob/master/4-Question_Answering/squad_example.png?raw=1)



The systems that can be used for that are listed on the official [Leaderboard](https://rajpurkar.github.io/SQuAD-explorer/) with human level of 89.4 F1 score

We are going to use [DeepPavlov SQuAD model](http://docs.deeppavlov.ai/en/master/features/models/squad.html#pretrained-models) based on BERT, achieving 88.4 F1 score. 

Dealing with SQuAD model includes:

0. Model installation and interaction with the pretrained model
0. Training the model on your own data
0. Skill example 

# Model installation and interaction

In [1]:
!pip install deeppavlov
!python -m deeppavlov install squad_bert

     |████████████████████████████████| 849kB 7.7MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 20.1MB 8.2MB/s 
     |████████████████████████████████| 7.3MB 23.1MB/s 
     |████████████████████████████████| 512kB 55.8MB/s 
     |████████████████████████████████| 860kB 50.4MB/s 
     |████████████████████████████████| 665kB 47.8MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 1.5MB 48.1MB/s 
     |████████████████████████████████| 8.0MB 27.0MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 10.4MB 52.9MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 6.7MB 20.9MB/s 
     |████████████████████████████████| 2.1MB 53.0MB/s 
     |

2020-07-09 16:39:48.408 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/squad/squad_bert.json'
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-w1q9yfk1
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-w1q9yfk1
  Running command git checkout -b feat/multi_gpu --track origin/feat/multi_gpu
  Switched to a new branch 'feat/multi_gpu'
  Branch 'feat/multi_gpu' set up to track remote branch 'feat/multi_gpu' from 'origin'.
  Created wheel for bert-dp: filename=bert_dp-1.0-cp36-none-any.whl size=23581 sha256=754be1e1b2f3639ef71852da46558469219dee0cba2e771ac496adb7346eed26
  Stored in directory: /tmp/pip-ephem-wheel-cache-ldclj6ya/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp
     |████████████████████████████████| 110.5MB 97kB/s 
     |███████████████████████████

In [2]:
# downloading the model
from deeppavlov import build_model, configs

model = build_model(configs.squad.squad, download=True)


2020-07-09 16:41:25.58 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_model_1.4_cpu_compatible.tar.gz to /root/.deeppavlov/squad_model_1.4_cpu_compatible.tar.gz
100%|██████████| 222M/222M [00:26<00:00, 8.24MB/s]
2020-07-09 16:41:52.124 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/squad_model_1.4_cpu_compatible.tar.gz archive into /root/.deeppavlov/models
2020-07-09 16:41:56.791 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M-char.vec to /root/.deeppavlov/downloads/embeddings/wiki-news-300d-1M-char.vec
100%|██████████| 7.73M/7.73M [00:01<00:00, 5.41MB/s]
2020-07-09 16:41:58.416 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M.vec to /root/.deeppavlov/downloads/embeddings/wiki-news-300d-1M.vec
100%|█████████

2020-07-09 16:47:48.110 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead 

2020-07-09 16:47:49.51 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 
2020-07-09 16:47:49.179 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 
2020-07-09 16:47:49.270 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-07-09 16:48:00.103 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/squad_model/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/squad_model/model


In [3]:
# using the pretrained model
model(['DeepPavlov is library for NLP and dialog systems.'], ['What is DeepPavlov?'])

[['library for NLP and dialog systems'], [14], [8040850.5]]

To use the model, you should pass a question and a corresponding text paragraph (where to find the answer)

You can also pass multiple questions, then the index of each question should correspond with the index of the paragraph:

In [7]:
question = 'What is neural network?'
text = '''Artificial neural networks or connectionist systems are computing systems vaguely inspired by the biological neural networks that constitute animal brains.[1]

The data structures and functionality of neural nets are designed to simulate associative memory. Neural nets learn by processing examples, each of which contains a known "input" and "result," forming probability-weighted asociations between the two, which are stored within the data structure of the net itself. (The "input" here is more accurately called an input set, since it generally consists of multiple independent variables, rather than a single value.) Thus, the "learning" of a neural net from a given example is the difference in the state of the net before and after processing the example. After being given a sufficient number of examples, the net becomes capable of predicting results from inputs, using the associations built from the example set. If a feedback loop is provided to the neural net about the accuracy of its predictions, it continues to refine its associations, resulting in an ever-increasing level of accuracy. In short, there is a direct relationship between the number and diversity of examples processed by a neural net and the accuracy of its predictions. This is why a neural net gets "better" with use. What is interesting about neural nets is that because they are indiscriminate in the way they form associations, they can form unexpected associations, and reveal relationships and dependencies that were not previously known.'''

model([text],[question])

[['computing systems vaguely inspired by the biological neural networks that constitute animal brains'],
 [56],
 [695200.625]]

Besides the answer itself, you also get the starting chat span and 

# Training the model 

**Important:** training with default config requires about 10Gb on GPU. 
Run following command to train the model:



In [ ]:
!python -m deeppavlov train deeppavlov/configs/squad/squad_bert.json

To understand wheater the model should be activated or not, we can 1) try to guess if the mmessage containf questions 2) use a classifier understanding is the question informative or not